<h2>Final Exam</h2>
You are a stock analyst working in the research department of a small<br>
hedge fund.  Your job is to analyze the returns of a pool of securities<br>
and back test the quants strategy of buys and sells.  You need<br>
to load the 4 securities and test their strategy.<br>
files:<br>
INTC - Intel Corporation<br>
AMD - Advanced Micro Devices<br>
AAPL - Apple<br>
FB - Meta Platforms, Inc.<br>
corporations = {'INTC': 'Intel Corporation',<br>
'AMD':'Advanced Micro Devices',<br>
'AAPL':'Apple',<br>
'FB':'Meta Platforms, Inc.'}<br>
<br>
NOTE: All trades use the closing price!!!!

<h2>Step 1</h2>
<h4>5 points</h4>
Create a directory and place all of the csv files into it<br>
Create the variable proj_dir = '  <your directory> '<br>
db_name = 'stock_analysis'

In [2]:
proj_dir = 'Exam_Files/'
db_name = 'stock_analysis.db'

<h2>Step 2</h2>
<h4>5 points</h4>
Create a database called 'Stock_Analysis'

In [3]:
# Create your database here
import sqlite3

connection = sqlite3.connect(db_name)
connection.close()

<h2>Step 3</h2>
<h4>15 points</h4>
Create the following tables:<br>
1 - security_master<br>
symbol<br>
trade_date<br>
open_price<br>
high_price<br>
low_price<br>
adj_close<br>
close_price<br>
volume<br>
<br>
2 - security_info<br>
symbol<br>
name  Max of 25 characters<br>
research_notes Max of 255 characters<br>
<br>
3 - security_log<br>
transaction_id  Character max 20 alpha numeric<br>
trade_date<br>
symbol<br>
trans_type   - (will contain only 'BUY' or 'SELL')<br>
shares   (Will only contain whole numbers)<br>
cost - Will hold dollar amounts<br>
balance - Will hold dollar amounts<br>

In [4]:
import sqlite3

connection = sqlite3.connect(db_name)
cursor = connection.cursor()

# Create security_master table
cursor.execute('create table if not exists security_master (symbol, trade_date, open_price, high_price, low_price, adj_close, close_price real, volume)')

# Create security_info table
cursor.execute('create table if not exists security_info (symbol, name VARCHAR(25), research_notes VARCHAR(255))')

# Create security_log table
cursor.execute('create table if not exists security_log (transaction_id VARCHAR(20), trade_date, symbol, trans_type text, shares int, cost real, balance real)')

connection.commit()
connection.close()

<h2>Step 4</h2>
<h4>15 points</h4>
Load the security master table<br>
Your code should read whatever files that are csv files in the directory<br>
Use the dictionary below to populate the security_info table:<br>
corporations = {'INTC': 'Intel Corporation',
'AMD':'Advanced Micro Devices',
'AAPL':'Apple',
'FB':'Meta Platforms, Inc.'}

In [5]:
import sqlite3



corporations = {'INTC': 'Intel Corporation',
'AMD':'Advanced Micro Devices',
'AAPL':'Apple',
'FB':'Meta Platforms, Inc.'}

connection = sqlite3.connect(db_name)
cursor = connection.cursor()

# Removing existing data, if it exists, in order to not duplicate data in table
cursor.execute('DELETE FROM security_master;')
cursor.execute('DELETE FROM security_info;')

# Iterate corporation name codes
for company_code in corporations:
    # -----------------------[LOAD SECURITY_MASTER TABLE]-----------------------
    # open each file using the directory, name, and extension
    with open(proj_dir + company_code + '.csv') as file:
        parsed_file_data = []
        for line in file:
            # turn line into sepearte values, with company code at the front
            line = [company_code] + line.strip().split(',')
            # add line to parsed file data
            parsed_file_data.append(line)
        # remove the description line from the file
        parsed_file_data = parsed_file_data[1:]
        # add the file contents to the database
        cursor.executemany('insert into security_master values(?,?,?,?,?,?,?,?)',parsed_file_data)



    # -----------------------[LOAD SECURITY_INFO TABLE]-----------------------
    company_name = corporations[company_code]
    cursor.execute('insert into security_info values(?,?,?)',(company_code,company_name,''))



# Commit updates and close connection
connection.commit()
connection.close()    

<h2>Step 5</h2>
<h4>10 points</h4>
Print the symbol,name, and number of records of every security using 1 SQL statement<br>

In [6]:
import sqlite3

connection = sqlite3.connect(db_name)
cursor = connection.cursor()

# 1 SQL statement, just repeated. Can't use select with executemany so I assume this is ok?
for name in corporations:
    cursor.execute(f"SELECT A.symbol, B.name, count(A.symbol) FROM security_master A LEFT JOIN security_info B on A.symbol == B.symbol where B.symbol == ?",(name,))
    # print the data per company
    print(*cursor.fetchall()[0])
    
connection.close()

INTC Intel Corporation 754
AMD Advanced Micro Devices 754
AAPL Apple 754
FB Meta Platforms, Inc. 754


<h2>Step 6</h2>
<h4>10 points</h4>
Print the average closing share price for each security in the year 2019<br>

In [7]:
import sqlite3

connection = sqlite3.connect(db_name)
cursor = connection.cursor()

# For every corporation name
for name in corporations:
    # retrieve the average closing price per company
    closing_price = list(cursor.execute("select avg(close_price) from security_master where strftime('%Y',trade_date)='2019' and symbol == ?",(name,)))
    # print the results
    print(f'Security: {name} \nYear: 2019 \nAverage Closing Price: {closing_price[0][0]}\n')
    
connection.close()

Security: INTC 
Year: 2019 
Average Closing Price: 48.324832607142824

Security: AMD 
Year: 2019 
Average Closing Price: 29.94107146031748

Security: AAPL 
Year: 2019 
Average Closing Price: 50.99662557936508

Security: FB 
Year: 2019 
Average Closing Price: 181.63749959523804



<h2>Step 7</h2>
<h4>5 points</h4>
Print the max and min closing share price for each security in the year 2020<br>

In [8]:
import sqlite3

connection = sqlite3.connect(db_name)
cursor = connection.cursor()

# For every corporation name
for name in corporations:
    # retrieve the average closing price per company
    closing_price = list(cursor.execute("select max(close_price), min(close_price) from security_master where strftime('%Y',trade_date)='2020' and symbol == ?",(name,)))
    # print the results
    print(f'Security: {name} \nYear: 2020 \nMax Closing Price: {closing_price[0][0]} \nMin Closing Price: {closing_price[0][1]}\n')
    
connection.close()

Security: INTC 
Year: 2020 
Max Closing Price: 65.120239 
Min Closing Price: 42.636486

Security: AMD 
Year: 2020 
Max Closing Price: 97.120003 
Min Closing Price: 38.709999

Security: AAPL 
Year: 2020 
Max Closing Price: 135.852524 
Min Closing Price: 55.402668

Security: FB 
Year: 2020 
Max Closing Price: 303.910004 
Min Closing Price: 146.009995



<h2>Step 8</h2>
<h4>15 points</h4>
Alter the trading strategy below to update the security_log table<br>
transaction_id = YYYYMMDD_symbol_random number, 100 up to 999<br>
trade_date <br>
symbol <br>
trans_type BUY OR SELL<br>
shares number of shares<br>
cost =  -1 * close_price * shares if a BUY, close_price * shares if a SELL<br>
<br>
<code>
sql = '''
SELECT symbol,trade_date,close_price - open_price as diff,close_price 
FROM security_master 
WHERE symbol = 'AMD'; '''
cursor.execute(sql)
rows = cursor.fetchall()
up_days = 0
down_days = 0
tot_val = 0
tot_shares = 0
share_size = 10000
for row in rows:
    symbol = row[0]
    y,m,d = str(row[1].year), ('00'+str(row[1].month))[-2:], ('00'+str(row[1].day))[-2:]
    transaction_id = f'{y}{m}{d}_{row[0]}_{str(random.randint(100,1000))}'
    trade_date = row[1]    
    '''
    if closed up for 5 consecutive days SELL 10000 shares
    sell if down for 5 consecute days '''
    if row[2] > 0:
        up_days += 1
        down_days = 0
    else:
        up_days = 0
        down_days += 1
    if up_days >= 5:
        down_days = 0
        '''SELL signal'''
        if tot_shares > 0:
            factor = 2 if tot_shares - (share_size*2) > 0 else 1
            tot_val += ((share_size*factor) * row[3])
            tot_shares -= (share_size*factor)
            # = YYYYMMDD_symbol_random number, 100 up to 999<br>
            print(f'****** SELL HERE {row[0]} {row[1]} {row[3]}  - {tot_shares} {tot_val}') 
            '''----- ADD CODE BELOW --------------'''
            
    if down_days >= 5:
        up_days = 0
        down_days = 0
        '''buy signal'''
        tot_val += (-1 * share_size * row[3])
        tot_shares += share_size
        print(f'****** BUY HERE {row[0]} {row[1]} {row[3]} - {tot_shares} {tot_val}')        
        '''----- ADD CODE BELOW --------------'''
</code>

In [9]:
import sqlite3
import random
import datetime






sql = '''
SELECT symbol,trade_date,close_price - open_price as diff,close_price 
FROM security_master 
WHERE symbol = 'AMD'; '''

connection = sqlite3.connect(db_name)
cursor = connection.cursor()

# ------------------ NEW CODE BEGIN ----------------
# Removing existing data, if it exists, in order to not duplicate data in table
cursor.execute('DELETE FROM security_log;')
connection.commit()
# ------------------ NEW CODE END ------------------

cursor.execute(sql)
rows = cursor.fetchall()

up_days = 0
down_days = 0
tot_val = 0
tot_shares = 0
share_size = 10000


for row in rows:

# ------------------ NEW CODE BEGIN ----------------
    row = list(row)
    symbol = row[0]
    row[1] = [int(item) for item in row[1].split('-')]
    row[1] = datetime.date(*row[1])
# ------------------ NEW CODE END ------------------
    
    y,m,d = str(row[1].year), ('00'+str(row[1].month))[-2:], ('00'+str(row[1].day))[-2:]
    transaction_id = f'{y}{m}{d}_{row[0]}_{str(random.randint(100,1000))}'
    trade_date = row[1]    
    '''
    if closed up for 5 consecutive days SELL 10000 shares
    sell if down for 5 consecute days '''
    if row[2] > 0:
        up_days += 1
        down_days = 0
    else:
        up_days = 0
        down_days += 1
    if up_days >= 5:
        down_days = 0
        '''SELL signal'''
        if tot_shares > 0:
            factor = 2 if tot_shares - (share_size*2) > 0 else 1
            tot_val += ((share_size*factor) * row[3])
            tot_shares -= (share_size*factor)
            # = YYYYMMDD_symbol_random number, 100 up to 999<br>
            print(f'****** SELL HERE {row[0]} {row[1]} {row[3]}  - {tot_shares} {tot_val}') 


            '''----- ADD CODE BELOW --------------'''
            connection = sqlite3.connect(db_name)
            cursor = connection.cursor()
            cursor.execute('''insert into security_log values(?,?,?,?,?,?,?)''',
            (transaction_id, trade_date, symbol, 'SELL', tot_shares, (share_size) * row[3], tot_val))
            connection.commit()
            connection.close()
            

    if down_days >= 5:
        up_days = 0
        down_days = 0
        '''buy signal'''
        tot_val += (-1 * share_size * row[3])
        tot_shares += share_size
        print(f'****** BUY HERE {row[0]} {row[1]} {row[3]} - {tot_shares} {tot_val}')    


        '''----- ADD CODE BELOW --------------'''
        connection = sqlite3.connect(db_name)
        cursor = connection.cursor()
        cursor.execute('''insert into security_log values(?,?,?,?,?,?,?)''',
        (transaction_id, trade_date, symbol, 'BUY', tot_shares, -((share_size) * row[3]), tot_val))
        connection.commit()
        connection.close()

****** BUY HERE AMD 2019-02-07 22.67 - 10000 -226700.00000000003
****** BUY HERE AMD 2019-03-28 25.059999 - 20000 -477299.99000000005
****** BUY HERE AMD 2019-06-25 28.860001 - 30000 -765900.0
****** SELL HERE AMD 2019-07-09 33.150002  - 10000 -102899.95999999996
****** SELL HERE AMD 2019-07-10 33.790001  - 0 235000.05
****** BUY HERE AMD 2019-08-06 28.860001 - 10000 -53599.96000000002
****** SELL HERE AMD 2019-11-14 38.349998  - 0 329900.01999999996
****** BUY HERE AMD 2020-05-01 49.880001 - 10000 -168899.99000000005
****** SELL HERE AMD 2020-09-30 81.989998  - 0 650999.99
****** BUY HERE AMD 2020-10-20 81.559998 - 10000 -164599.99
****** BUY HERE AMD 2020-12-22 93.160004 - 20000 -1096200.03
****** BUY HERE AMD 2021-01-19 89.449997 - 30000 -1990700.0
****** BUY HERE AMD 2021-01-28 87.519997 - 40000 -2865899.97
****** BUY HERE AMD 2021-02-22 85.370003 - 50000 -3719600.0
****** BUY HERE AMD 2021-03-08 73.959999 - 60000 -4459199.99
****** BUY HERE AMD 2021-05-03 78.550003 - 70000 -524470

<h2>Step 9</h2>
<h4>10 points</h4>
<div style='display: none;'>
If you missed the update on step 8 above manual run the following inserts<br>
Then 
</div>
return the symbol,stock price,trans_type For all 2020 trades<br>

<div style='display: none;'>

INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20190207_AMD_205','2019-02-07','AMD','BUY',
        10000,-226700.0,-226700.0)

INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20190328_AMD_354','2019-03-28','AMD','BUY',
        20000,-250599.99,-477299.99)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20190625_AMD_336','2019-06-25','AMD','BUY',
        30000,-288600.01,-765900.0)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20190709_AMD_348','2019-07-09','AMD','SELL',
            10000,331500.02,-102899.96)
            
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20190710_AMD_191','2019-07-10','AMD','SELL',
            0,337900.01,235000.05)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20190806_AMD_645','2019-08-06','AMD','BUY',
        10000,-288600.01,-53599.96)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20191114_AMD_174','2019-11-14','AMD','SELL',
            0,383499.98,329900.02)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20200501_AMD_727','2020-05-01','AMD','BUY',
        10000,-498800.01,-168899.99)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20200930_AMD_691','2020-09-30','AMD','SELL',
            0,819899.98,650999.99)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20201020_AMD_135','2020-10-20','AMD','BUY',
        10000,-815599.98,-164599.99)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20201222_AMD_392','2020-12-22','AMD','BUY',
        20000,-931600.04,-1096200.03)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210119_AMD_612','2021-01-19','AMD','BUY',
        30000,-894499.97,-1990700.0)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210128_AMD_454','2021-01-28','AMD','BUY',
        40000,-875199.97,-2865899.97)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210222_AMD_258','2021-02-22','AMD','BUY',
        50000,-853700.03,-3719600.0)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210308_AMD_803','2021-03-08','AMD','BUY',
        60000,-739599.99,-4459199.99)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210503_AMD_543','2021-05-03','AMD','BUY',
        70000,-785500.03,-5244700.02)

INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210528_AMD_974','2021-05-28','AMD','SELL',
            50000,800800.02,-3643099.98)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210716_AMD_346','2021-07-16','AMD','BUY',
        60000,-858899.99,-4501999.97)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210723_AMD_399','2021-07-23','AMD','SELL',
            40000,921500.02,-2658999.93)
            
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210803_AMD_858','2021-08-03','AMD','SELL',
            20000,1125599.98,-407799.97)
            
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210804_AMD_569','2021-08-04','AMD','SELL',
            10000,1187699.97,779900.0)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210913_AMD_796','2021-09-13','AMD','BUY',
        20000,-1048000.03,-268100.03)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210927_AMD_123','2021-09-27','AMD','SELL',
            10000,1081600.04,813500.01)
</div>

In [10]:
import sqlite3

connection = sqlite3.connect(db_name)
cursor = connection.cursor()


# retrieve the transactions for 2020 and match the trade_date of both tables to get the share price on the transaction day
closing_price = list(cursor.execute('''select A.symbol, A.trade_date, B.close_price, A.trans_type from security_log A LEFT JOIN 
security_master B on A.trade_date == B.trade_date where strftime('%Y', A.trade_date)=='2020' and A.symbol == B.symbol'''))


# print the transactions
for transaction in closing_price:
    print(*transaction)
    
connection.close()

AMD 2020-05-01 49.880001 BUY
AMD 2020-09-30 81.989998 SELL
AMD 2020-10-20 81.559998 BUY
AMD 2020-12-22 93.160004 BUY


<h2>Step 10</h2>
<h4>10 points</h4>
If you bought 100,000 shares of FB the first trading day of 2020 and sold<br>
100,000 shares on the last day of 2020. How much would you have gained or lost?

In [11]:
import sqlite3

connection = sqlite3.connect(db_name)
cursor = connection.cursor()


# retrieve the transactions for 2020
stock_history = list(cursor.execute("select symbol, trade_date, close_price from security_master where strftime('%Y',trade_date)=='2020' and symbol == 'FB' order by trade_date"))

first_day_2020_close_price = float(stock_history[0][2])
last_day_2020_close_price = float(stock_history[-1][2])

# Opening price 

print(first_day_2020_close_price, last_day_2020_close_price)

# The difference of the buy and sell prices for the first and last day of 2020 for FB
print(f'Potential Gain: ${round(abs(first_day_2020_close_price*100000 - last_day_2020_close_price*100000),2)}')


connection.close()

209.779999 273.160004
Potential Gain: $6338000.5


<h2>Bonus Question</h2>
<h4>25 points</h4>
Write a program that buys 10000 shares of INTC every time the volume<br>
is greater than 10% for the average daily volume for 10 days and sell 10000 shares <br>
when the average volume for the past 5 days.  The initial buy should be when the shares <br>
have had 5 consecutive positive trading days

In [17]:
import sqlite3
import random
import datetime
import statistics

sql = '''
SELECT symbol,trade_date,close_price - open_price as diff,close_price,volume 
FROM security_master 
WHERE symbol = 'AMD'; '''

connection = sqlite3.connect(db_name)
cursor = connection.cursor()

# ------------------ NEW CODE BEGIN ----------------
# Removing existing data, if it exists, in order to not duplicate data in table
connection.commit()
# ------------------ NEW CODE END ------------------

cursor.execute(sql)
rows = cursor.fetchall()

up_days = 0
down_days = 0
tot_val = 0
tot_shares = 0
share_size = 10000


# the index from enumerate will be used in order to get the average of the 10 previous days
for index, row in enumerate(rows):


# ------------------ NEW CODE BEGIN ----------------
    row = list(row)
    symbol = row[0]
    row[1] = [int(item) for item in row[1].split('-')]
    row[1] = datetime.date(*row[1])
# ------------------ NEW CODE END ------------------
    
    y,m,d = str(row[1].year), ('00'+str(row[1].month))[-2:], ('00'+str(row[1].day))[-2:]
    transaction_id = f'{y}{m}{d}_{row[0]}_{str(random.randint(100,1000))}'
    trade_date = row[1]    
    '''
    if closed up for 5 consecutive days SELL 10000 shares
    sell if down for 5 consecute days '''
    if row[2] > 0:
        up_days += 1
        down_days = 0
    else:
        up_days = 0
        down_days += 1


    if up_days >= 5:
        down_days = 0
        '''SELL signal'''
        #  INSERTED CODE
        # print(row)
        # [print(rows[index-number-1]) for number in range(10)]

        # ten_day_average = [int(rows[index-number-1][-1]) for number in range(5)]
        # print(f'10 day average {statistics.mean(ten_day_average)}')
        # # print('\n\n')
        # if float(row[-1]) < statistics.mean(ten_day_average) + statistics.mean(ten_day_average)*.1:
        #     break


        if tot_shares > 0:
            factor = 2 if tot_shares - (share_size*2) > 0 else 1
            tot_val += ((share_size*factor) * row[3])
            tot_shares -= (share_size*factor)
            # = YYYYMMDD_symbol_random number, 100 up to 999<br>
            print(f'****** SELL HERE {row[0]} {row[1]} {row[3]}  - {tot_shares} {tot_val}') 


            '''----- ADD CODE BELOW --------------'''
            

    if down_days >= 5:

        #  INSERTED CODE
        # up_days = 0
        # down_days = 0
        # '''buy signal'''

        # ten_day_average = [int(rows[index-number-1][-1]) for number in range(10)]
        # print(f'10 day average {statistics.mean(ten_day_average)}')

        if float(row[-1]) < statistics.mean(ten_day_average) + statistics.mean(ten_day_average)*.1:
            break


        tot_val += (-1 * share_size * row[3])
        tot_shares += share_size
        print(f'****** BUY HERE {row[0]} {row[1]} {row[3]} - {tot_shares} {tot_val}')    


        '''----- ADD CODE BELOW --------------'''


10 day average 104915660
